In [1]:
%store -r finalDocumentSet

In [2]:
print(len(finalDocumentSet))

55


In [24]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Pooja
[nltk_data]     Ajit\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [25]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [10]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [26]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])


In [9]:
%store -r cleanDocx

In [10]:
print(len(cleanDocx))

55


In [45]:
%store -r dfObj1

In [3]:
print(len(dfObj1))

88


In [46]:
dfObj1

,0
0,"[""['health planners policy-makers face comple..."
1,"[""['knowledge organization () (also termed inf..."
2,"[""['searching broad topic areas raises particu..."
3,"[""[' systematic reviews seek answer spec..."
4,"[""[': systematic reviews provide less biased ..."
...,...
83,"[""[' introduced many ways which -demics p..."
84,"[""[': meta-research studies investigating meth..."
85,"[""[' systematic reviews.london: sage; 2012.go..."
86,"[""['*correspondence : . wettermark, centre ph..."


In [47]:
# Convert to list
#data = dfObj.tolist()

# Use ravel to convert 2-d to 1-d array
data = dfObj1.values.tolist()



In [49]:
print(data[0:1])

[['["[\'health planners  policy-makers face complex decisions requiringa deep knowledge  ‘healthcare systems’, which consist    -ganizations, people  actions involved  maintaining, restoring andenhancing human well-being.", \' characteristics  healthcare systemsare considered   important indicator  population health (..,quality  life  people  developing integrated smart systems forcities  regions) (ismagilova, hughes, dwivedi, & raman, 2019).\', \'thereis signiﬁcant variation  policy, funding  delivery  health careacross regions (griﬃn  ., 2016).\', \'understanding these diﬀerencesand  they impact   health  communities requires reliableevidence,  health planners   this evidence  drive systematicquality improvement  regional healthcare delivery. growing availability  healthcare data raises  prospect ofbetter evidence-informed decision-making.\', \'healthcare systems involveandchange,constantvariabilitycomplex interactions between structures, processes, outcomes andagents.\', \'they  cha

In [50]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [51]:
data_words = list(sent_to_words(data))


In [52]:
print(data_words[:1])

[['health', 'planners', 'policy', 'makers', 'face', 'complex', 'decisions', 'requiringa', 'deep', 'knowledge', 'healthcare', 'systems', 'which', 'consist', 'ganizations', 'people', 'actions', 'involved', 'maintaining', 'restoring', 'andenhancing', 'human', 'well', 'being', 'characteristics', 'healthcare', 'systemsare', 'considered', 'important', 'indicator', 'population', 'health', 'quality', 'life', 'people', 'developing', 'integrated', 'smart', 'systems', 'forcities', 'regions', 'ismagilova', 'hughes', 'dwivedi', 'raman', 'thereis', 'signiﬁcant', 'variation', 'policy', 'funding', 'delivery', 'health', 'careacross', 'regions', 'griﬃn', 'understanding', 'these', 'diﬀerencesand', 'they', 'impact', 'health', 'communities', 'requires', 'health', 'planners', 'this', 'evidence', 'drive', 'improvement', 'regional', 'healthcare', 'delivery', 'growing', 'availability', 'healthcare', 'data', 'raises', 'prospect', 'ofbetter', 'evidence', 'informed', 'decision', 'making', 'healthcare', 'systems',

In [53]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])


['health', 'planners', 'policy_makers', 'face', 'complex', 'decisions', 'requiringa', 'deep', 'knowledge', 'healthcare', 'systems', 'which', 'consist', 'ganizations', 'people', 'actions', 'involved', 'maintaining', 'restoring', 'andenhancing', 'human', 'well', 'being', 'characteristics', 'healthcare', 'systemsare', 'considered', 'important', 'indicator', 'population', 'health', 'quality', 'life', 'people', 'developing', 'integrated', 'smart', 'systems', 'forcities', 'regions', 'ismagilova', 'hughes', 'dwivedi', 'raman', 'thereis', 'signiﬁcant', 'variation', 'policy', 'funding', 'delivery', 'health', 'careacross', 'regions', 'griﬃn', 'understanding', 'these', 'diﬀerencesand', 'they', 'impact', 'health', 'communities', 'requires', 'health', 'planners', 'this', 'evidence', 'drive', 'improvement', 'regional', 'healthcare', 'delivery', 'growing', 'availability', 'healthcare', 'data', 'raises', 'prospect', 'ofbetter', 'evidence', 'informed_decision_making', 'healthcare', 'systems', 'interact

In [54]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out


In [16]:
pip install spacy en

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [111]:
pip install spacy


Note: you may need to restart the kernel to use updated packages.


In [18]:
!python -m spacy download en

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
[x] Couldn't link model to 'en'
Creating a symlink in spacy/data failed. Make sure you have the required
permissions and try re-running the command as admin, or use a virtualenv. You
can still import the model as a module and call its load() method, or create the
symlink manually.
F:\python\lib\site-packages\en_core_web_sm -->
F:\python\lib\site-packages\spacy\data\en
[!] Download successful but linking failed
Creating a shortcut link for 'en' didn't work (maybe you don't have admin
permissions?), but you can still load the model via its full package name: nlp =
spacy.load('en_core_web_sm')


You should consider upgrading via the 'python -m pip install --upgrade pip' command.
You do not have sufficient privilege to perform this operation.


In [55]:
nlp = spacy.load('en_core_web_sm')

In [56]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
#python3 -m spacy download en
#nlp = spacy.download('en')
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])


[['health', 'planner', 'face', 'complex', 'decision', 'requiringa', 'deep', 'system', 'consist', 'ganization', 'people', 'action', 'involve', 'maintain', 'restore', 'andenhance', 'human', 'consider', 'important', 'indicator', 'population', 'quality', 'life', 'people', 'develop', 'integrate', 'smart', 'system', 'forcitie', 'region', 'signiﬁcant', 'policy', 'funding', 'delivery', 'health', 'careacross', 'region', 'griﬃn', 'understand', 'impact', 'health', 'community', 'require', 'health', 'planner', 'evidence', 'drive', 'improvement', 'regional', 'healthcare', 'delivery', 'grow', 'availability', 'raise', 'ofbetter', 'evidence', 'inform', 'healthcare', 'system', 'interaction', 'structure', 'process', 'outcome', 'andagent', 'characterize', 'uncertainty', 'meadow', 'mental', 'care', 'epitomize', 'thiscomplexity', 'mental', 'illness', 'often', 'health', 'social', 'dimension', 'planner', 'todrive', 'quality', 'improvement', 'useful', 'datum', 'must', 'reﬂect', 'complexity', 'ofmental', 'healt

In [58]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])


[[(0, 1), (1, 3), (2, 2), (3, 1), (4, 5), (5, 2), (6, 2), (7, 2), (8, 1), (9, 2), (10, 2), (11, 2), (12, 2), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 2), (20, 2), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 14), (30, 1), (31, 1), (32, 1), (33, 2), (34, 1), (35, 1), (36, 2), (37, 1), (38, 19), (39, 3), (40, 6), (41, 5), (42, 2), (43, 2), (44, 1), (45, 2), (46, 1), (47, 1), (48, 3), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 6), (55, 1), (56, 6), (57, 12), (58, 12), (59, 1), (60, 2), (61, 7), (62, 1), (63, 1), (64, 1), (65, 3), (66, 1), (67, 1), (68, 4), (69, 3), (70, 1), (71, 2), (72, 2), (73, 1), (74, 1), (75, 1), (76, 5), (77, 4), (78, 9), (79, 6), (80, 4), (81, 1), (82, 1), (83, 9), (84, 2), (85, 1), (86, 2), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 5), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 2), (103, 1), (104, 1), (105, 4), (106, 5), (107, 19), (108, 2), (109, 1), (11

In [59]:
id2word[0]

'aaron'

In [60]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('aaron', 1),
  ('ability', 3),
  ('able', 2),
  ('absence', 1),
  ('abstract', 5),
  ('academic', 2),
  ('access', 2),
  ('accord', 2),
  ('accuracy', 1),
  ('action', 2),
  ('actually', 2),
  ('acute', 2),
  ('additional', 2),
  ('address', 2),
  ('adecision', 1),
  ('admission', 1),
  ('adopt', 1),
  ('advance', 1),
  ('advanced', 2),
  ('age', 2),
  ('agency', 2),
  ('aggregated', 1),
  ('agreement', 2),
  ('ahead', 1),
  ('aim', 1),
  ('allocation', 1),
  ('allow', 2),
  ('alluvial', 1),
  ('already', 1),
  ('also', 14),
  ('alsoappeare', 1),
  ('alsorequire', 1),
  ('alter', 1),
  ('amazoniﬁcation', 2),
  ('ambiguity', 1),
  ('amethod', 1),
  ('analyse', 2),
  ('analysedusing', 1),
  ('analysis', 19),
  ('analytic', 3),
  ('analytical', 6),
  ('analyze', 5),
  ('andagent', 2),
  ('andanalysis', 2),
  ('andcommunicate', 1),
  ('andenhance', 2),
  ('andgap', 1),
  ('andiii', 1),
  ('andmf', 3),
  ('andprovide', 2),
  ('andpublic', 1),
  ('andservice', 1),
  ('apparent', 1),
  ('a

In [61]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [62]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.020*"review" + 0.013*"study" + 0.011*"use" + 0.011*"systematic" + '
  '0.009*"reference" + 0.009*"search" + 0.007*"system" + 0.007*"screening" + '
  '0.006*"model" + 0.006*"record"'),
 (1,
  '0.036*"case" + 0.018*"study" + 0.013*"paper" + 0.013*"software" + '
  '0.010*"use" + 0.008*"quality" + 0.008*"research" + 0.007*"requirement" + '
  '0.007*"test" + 0.007*"model"'),
 (2,
  '0.024*"study" + 0.021*"review" + 0.011*"systematic" + 0.010*"search" + '
  '0.009*"use" + 0.007*"research" + 0.007*"include" + 0.006*"approach" + '
  '0.006*"result" + 0.006*"base"'),
 (3,
  '0.019*"study" + 0.017*"review" + 0.016*"evidence" + 0.012*"systematic" + '
  '0.008*"abstract" + 0.007*"report" + 0.006*"intervention" + 0.006*"primary" '
  '+ 0.006*"outcome" + 0.005*"clinical"'),
 (4,
  '0.017*"application" + 0.015*"study" + 0.013*"smell" + 0.013*"software" + '
  '0.011*"base" + 0.011*"process" + 0.010*"use" + 0.008*"datum" + '
  '0.007*"attack" + 0.007*"mining"'),
 (5,
  '0.024*"trial" + 0.023*

In [63]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Perplexity:  -7.703940116336915

Coherence Score:  0.3871962384755186


In [64]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

f:\python\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.048036 -0.008836       1        1  30.871353
8     -0.103554 -0.116481       2        1  10.115047
0      0.057471 -0.001493       3        1   9.920610
4     -0.058302  0.076085       4        1   9.575662
7      0.097926 -0.028958       5        1   8.306994
6     -0.039665 -0.074572       6        1   6.884695
9     -0.057704 -0.003080       7        1   6.692848
5      0.069691  0.043025       8        1   6.660535
1     -0.088132  0.116403       9        1   5.604925
3      0.074233 -0.002093      10        1   5.367327, topic_info=    Category         Freq      Term        Total  loglift  logprob
935  Default   800.000000      case   800.000000  30.0000  30.0000
834  Default   572.000000     trial   572.000000  29.0000  29.0000
658  Default  2792.000000    review  2792.000000  28.0000  28.0000
679  Default  1766.000000    search  1766.000000  27.0000  27.0000
272  Default   732.000000  evidence   732.000000  26.0000  26.0000
..       ...          ...       ...          ...      ...      ...
647  Topic10    48.786503  research  1260.990234  -0.3274  -5.4814
655  Topic10    44.284924    result  1225.727051  -0.3958  -5.5782
58   Topic10    40.792240  approach   888.078613  -0.1557  -5.6603
182  Topic10    39.533768     datum  1022.865967  -0.3284  -5.6917
488  Topic10    38.863380    method   970.963562  -0.2934  -5.7088

[764 rows x 6 columns], token_table=       Topic      Freq             Term
term                                   
2336       3  0.917888             aait
1710       3  0.078107        abstrackr
1710       4  0.019527        abstrackr
1710       5  0.878700        abstrackr
4          1  0.267403         abstract
...      ...       ...              ...
8664       5  0.956608  workload_saving
18638      1  0.970236           writer
11460      7  0.758565             xxxx
15117      4  0.985707              ﬂaw
3809       9  0.943424              ﬂow

[2284 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 9, 1, 5, 8, 7, 10, 6, 2, 4])

In [37]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)





In [38]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


In [76]:
df_dominant_topic.head(38)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,2.0,0.9947,"study, review, use, citation, datum, set, mode...","[health, planner, face, complex, decision, req..."
1,1,4.0,0.9996,"review, search, study, systematic, trial, case...","[also, term, research, science, concerned, oth..."
2,2,6.0,0.7263,"search, use, review, study, application, term,...","[search, broad, topic, area, raise, particular..."
3,3,0.0,0.9094,"study, base, review, use, system, result, scre...","[systematic, review, seek, answer, specific, r..."
4,4,4.0,0.5735,"review, search, study, systematic, trial, case...","[systematic, review, may, provide, less, biase..."
5,5,3.0,0.8066,"study, use, review, base, leaf, image, shape, ...","[systematic, literature, review, method, resea..."
6,6,4.0,0.9963,"review, search, study, systematic, trial, case...","[engineer, develop, software, thatinclude, des..."
7,7,3.0,0.9893,"study, use, review, base, leaf, image, shape, ...","[sentiment, opinion, currently, widely, studie..."
8,8,4.0,0.3781,"review, search, study, systematic, trial, case...","[technology, focus, generation, presentationan..."
9,9,8.0,0.9998,"citation, review, address, use, systematic, re...","[business, constant, work, utilize, technology..."


In [75]:
df_dominant_topic.tail(50)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
38,38,5.0,0.6722,"study, smell, software, pattern, research, bas...","[abbreviation, use, review, reference, researc..."
39,39,1.0,0.9999,"study, use, review, research, include, base, r...","[research, topicevidence, new, method, perform..."
40,40,6.0,0.6142,"search, use, review, study, application, term,...","[medical, practice, research, high, level, ev_..."
41,41,6.0,0.9116,"search, use, review, study, application, term,...","[automate, classification, study, promise, opt..."
42,42,0.0,0.9983,"study, base, review, use, system, result, scre...","[consist, complete, alveolar, severe, type, de..."
43,43,0.0,0.9999,"study, base, review, use, system, result, scre...","[ﬂourishe, way, toelicit, new, exciting, consu..."
44,44,4.0,1.0000,"review, search, study, systematic, trial, case...","[ongoing, support, include, grey_literature, s..."
45,45,1.0,0.9848,"study, use, review, research, include, base, r...","[crowdsource, allow, access, large, pool, part..."
46,46,2.0,0.6742,"study, review, use, citation, datum, set, mode...","[type, diabete, common, chronicdisease, world,..."
47,47,1.0,0.8994,"study, use, review, research, include, base, r...","[finding, document, people, improve, participa..."
